In [1]:
import geopandas as gpd
import utils.analysis as an

/Users/lcuartase/.conda/envs/ox_eafit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Zonas Medellin

In [2]:
zo = ['moravia_LH','floresta_HH','aguacatala_HL']
# z = zo[0]
for z in zo:
    print(f"Procesando zona: {z}")
    nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
    blocks = gpd.read_file(f'../data/output/shape/blocks/medellin_blocks.shp')

    blocks = blocks[['MANZ_CCNCT','pop','geometry']]
    nodes = nodes.to_crs('epsg:32618')
    blocks = blocks.to_crs('epsg:32618')

    # Create the voronoi polygons
    nodes = nodes.set_index('osmid')
    voronoi = an.voronoi_polygons(nodes)
    voronoi.to_crs('epsg:4326').to_file(f'../data/output/shape/physical_variables/population/voronoi/{z}/{z}_voronoi.shp')

    # Assing the population to each voronoi polygon
    blocks = blocks.set_index('MANZ_CCNCT')
    voronoi = an.assing_blocks_attribute_to_voronoi(blocks, voronoi, 'pop')

    # Calculate the population density in hab/hec
    voronoi['den_pop'] = voronoi['pop']/(voronoi.area/10000)

    # Merge the population with the nodes
    nodes = nodes.merge(voronoi[['osmid','den_pop']].set_index('osmid'), left_index=True, right_index=True, how='left')

    # Assing the population to the edges with an average
    for i in edges.index:
        u = edges.loc[i,'u']
        v = edges.loc[i,'v']
        edges.loc[i,'den_pop'] = (nodes.loc[u,'den_pop']+ nodes.loc[v,'den_pop'])/2
        
    # Export results
    nodes = nodes.reset_index()
    nodes = nodes.to_crs('epsg:4326')
    nodes.to_file(f'../data/output/shape/physical_variables/population/{z}/{z}_population_nodes.shp')
    edges.to_file(f'../data/output/shape/physical_variables/population/{z}/{z}_population_edges.shp')
    
    # Limit the study area to the neighborhood polygon
    edges = edges.reset_index()
    edges = edges.to_crs('epsg:32618')
    nodes = nodes.reset_index()
    nodes = nodes.to_crs('epsg:32618')

    if z == 'floresta_HH':
        w = 'Alto_Alta'
    elif z == 'moravia_LH':
        w = 'Bajo_Alta'
    elif z == 'aguacatala_HL':
        w = 'Alto_Baja'

    # Load polygon files
    polygon = gpd.read_file(f'../data/input/shape/Poligonos/{w}.shp')
    polygon = polygon.to_crs('epsg:32618')
    polygon = polygon['geometry'].union_all()
    polygon = gpd.GeoDataFrame(geometry = [polygon], crs = 'epsg:32618')

    # Select the segments that are inside the polygon
    for idx, row in polygon.iterrows():
        # Filter the edges contained in the polygon
        edges_in_polygon = edges[edges['geometry'].intersects(row['geometry'])]

    # List of the nodes inside the polygon based on the 'u' and 'v' columns of the edges inside the polygon
    nodes_in_polygon = edges_in_polygon['u'].to_list() + edges_in_polygon['v'].to_list()
    # Make the nodes unique and sort it
    nodes_in_polygon = list(set(nodes_in_polygon))
    nodes_in_polygon.sort()
    # Select the nodes inside the buffer
    nodes_in_polygon = nodes.set_index('osmid').loc[nodes_in_polygon].reset_index()

    # Export
    nodes_in_polygon = nodes_in_polygon.to_crs('epsg:4326')
    edges_in_polygon = edges_in_polygon.to_crs('epsg:4326')
    nodes_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_population_nodes.shp')
    edges_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_population_edges.shp')

Procesando zona: moravia_LH
Procesando zona: floresta_HH
Procesando zona: aguacatala_HL


# Medellin

In [3]:
z = 'medellin'

nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
blocks = gpd.read_file(f'../data/output/shape/blocks/medellin_blocks.shp')

blocks = blocks[['MANZ_CCNCT','pop','geometry']]
nodes = nodes.to_crs('epsg:32618')
blocks = blocks.to_crs('epsg:32618')


nodes['osmid'] = nodes['osmid'].astype(str)
edges['u'] = edges['u'].astype(str)
edges['v'] = edges['v'].astype(str)


# Create the voronoi polygons
nodes = nodes.set_index('osmid')
voronoi = an.voronoi_polygons(nodes)
voronoi.to_crs('epsg:4326').to_file(f'../data/output/shape/physical_variables/population/voronoi/{z}/{z}_voronoi.shp')

# Assing the population to each voronoi polygon
blocks = blocks.set_index('MANZ_CCNCT')
voronoi = an.assing_blocks_attribute_to_voronoi(blocks, voronoi, 'pop')

# Calculate the population density in hab/hec
voronoi['den_pop'] = voronoi['pop']/(voronoi.area/10000)

# Merge the population with the nodes
nodes = nodes.merge(voronoi[['osmid','den_pop']].set_index('osmid'), left_index=True, right_index=True, how='left')

# Assing the population to the edges with an average
for i in edges.index:
    u = edges.loc[i,'u']
    v = edges.loc[i,'v']
    edges.loc[i,'den_pop'] = (nodes.loc[u,'den_pop']+ nodes.loc[v,'den_pop'])/2
    
# Export results
nodes = nodes.reset_index()
nodes = nodes.to_crs('epsg:4326')


nodes['osmid'] = nodes['osmid'].astype(int)
edges['u'] = edges['u'].astype(int)
edges['v'] = edges['v'].astype(int)


nodes.to_file(f'../data/output/shape/physical_variables/population/{z}/{z}_population_nodes.shp')
edges.to_file(f'../data/output/shape/physical_variables/population/{z}/{z}_population_edges.shp')
